## Python Crawler to scrap linkedin recent posts/activity of a user

In [1]:
### Importing required libraries
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs4
import re

In [2]:
### browser params for selenium
firefox_options = Options()
firefox_options.add_argument("--incognito")
firefox_options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe' ## give firefox exe path here


In [3]:
### running the webdriver
driver = webdriver.Firefox(options=firefox_options, executable_path=r"..\driver\geckodriver.exe") ## path where driver is present

C:\Users\Yusra Shahid\AppData\Local\Temp\ipykernel_25572\501550139.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(options=firefox_options, executable_path=r"..\driver\geckodriver.exe") ## path where driver is present


In [4]:
import time
url = "https://linkedin.com/"
driver.get(url)
time.sleep(2)

In [5]:
### sign in to linkedin
signInButton = driver.find_element(By.XPATH,"/html/body/main/section[1]/div/div/form/button")
signInButton.click()

In [ ]:
USERNAME = input("Enter the username: ")
PASSWORD = input("Enter the password: ")
print(USERNAME)


In [7]:
email = driver.find_element(By.XPATH,'//*[@id="session_key"]')
email.send_keys(USERNAME)

password = driver.find_element(By.XPATH,'//*[@id="session_password"]')
password.send_keys(PASSWORD)

In [8]:
login = driver.find_element(By.XPATH,'/html/body/main/section[1]/div/div/form/button')
login.click()

In [9]:
### in case there is a phone verification setup page this will skip it
try:
    skip = driver.find_element(By.XPATH,'/html/body/div[2]/div[1]/section/div[2]/div/article/footer/div/div/button[2]')
    skip.click()
except:
    print("skip button not found")

skip button not found


In [10]:
### goto profile and then recent activity link
profile = driver.find_element(By.XPATH,'/html/body/div[5]/div[3]/div/div/div[2]/div/div/div/div[1]/div[1]/a/div[2]')
profile.click()

In [11]:
activity = driver.find_elements(By.XPATH,'/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[6]/div[3]/div/div/a')
for my_href in activity:
    activity_link = my_href.get_attribute("href")

In [12]:
driver.get(activity_link)
time.sleep(5)

#### getting posts that are gathered in 20 seconds of scroll

In [13]:
start=time.time()
n =20
lastHeight = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    newHeight = driver.execute_script("return document.body.scrollHeight")
    if newHeight == lastHeight:
        break
    lastHeight = newHeight
    end=time.time()
    if round(end-start)>n:
        break

In [14]:
#posts = driver.find_elements(By.CLASS_NAME,"scaffold-finite-scroll__content")
posts_source = driver.page_source 
linkedin_soup = bs4(posts_source.encode("utf-8"), "html")
linkedin_soup.prettify()
containers = linkedin_soup.findAll("div",{"class":"ember-view occludable-update"})

In [15]:
p_text,urls =[],[]

In [16]:
for container in containers:

    try:
        text_box = container.find("div",{"class":"feed-shared-update-v2__description-wrapper"})
        text = text_box.find("span",{"dir":"ltr"})
        post_text = text.text.strip()
        p_text.append(post_text)
        #print(post_text)
        if "https" in post_text:
            post_url = re.search("(?P<url>https?://[^\s]+)", post_text).group("url")
        else:
            post_url = ""
        #print(post_url)
        urls.append(post_url)
    except:
        #print(text_box)
        pass



In [23]:
print("total number of posts are: ",len(p_text))
print("total number of urls are: ",len(urls)-urls.count(""))

total number of posts are:  24
total number of urls are:  7


In [19]:
p_text

['🚀 Want to build a fast and scalable MVP for your business or startup? 📈🤔 Not sure whether to use ReactJS or Flutter? 💻📱👨\u200d💻 Join our webinar tomorrow with Flutter experts to learn about the pros and cons of each technology and make an informed decision for your digital transformation. 💡Speaker: Ishaq Hassan📅 Click here to join webinar: https://lnkd.in/d5bgi4Ab#ReactJS #Flutter #MVP #Startups #BusinessOwners #Webinar #TechDecisionMaking #DigitalTransformation #Scalability #startup #technology #business',
 'In my Advanced Computer Vision class at UCF this semester, we are discussing latest papers on Diffusion Models, it has been very interesting to notice how fast the field is progressing: https://lnkd.in/eEmTTG-WVideos at: https://lnkd.in/e2uQ_JGj',
 "Are you a business owner or an employer? Listen up! Here's what the future of work looks like:Remote Work: Remote work is no longer just a perk or an option, but a necessity in today's world. Companies are investing in the infrastruc

In [22]:
urls

['https://lnkd.in/d5bgi4Ab#ReactJS',
 'https://lnkd.in/eEmTTG-WVideos',
 '',
 '',
 '',
 '',
 '',
 'https://lnkd.in/gvHQDN5n',
 'https://lnkd.in/gMrCa7dcLink',
 '',
 '',
 'https://lnkd.in/gCVKv2xV#hotstar',
 '',
 '',
 '',
 '',
 'https://lnkd.in/eHRUsGsd',
 '',
 '',
 'https://lnkd.in/epripz9X',
 '',
 '',
 '',
 '']